# Beckhoff Automation Interface Example
This example shows a few scenarios for automation interface usage. To run install the .NET Interactive Notebooks extension for Visual Studio Code from https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode and then open this notebook in Visual Studio Code.

In [ ]:
# Load the COM message filter. See the Automation Interface documentation for more information.
Add-Type -TypeDefinition (Get-Content ".\MessageFilter.cs" -Raw)
[EnvDteUtils.MessageFilter]::Register()

Add-Type -Path "C:\TwinCAT\3.1\Components\Plc\Common\OnlineCommands.dll"
$loginServiceFlags = [_3S.CoDeSys.OnlineCommands.LoginServiceFlags]

In [ ]:
# Set the visual studio program ID. See https://infosys.beckhoff.com/english.php?content=../content/1033/tc3_automationinterface/242746251.html&id= for a list.
$vsName = "TcXaeShell.DTE.15.0"

# Set the project template, this one is provided by Beckhoff
$template = "C:\TwinCAT\3.1\Components\Base\PrjTemplate\TwinCAT Project.tsproj"

# Set the plc template, this one is provided by Beckhoff
$plcTemplate = "Standard PLC Template"

# Configure where to place the generated project
$currentDir = Get-Location
$projectDirectory = "$currentDir\project"
$projectName = "TestPLC"
$plcName = "TestPLC"

# Configure where to deploy the generated project to. This could be local host (127.0.0.1) or a remote PLC
$targetIpAddress = "10.1.12.180"

In [ ]:
# Access the Visual Studio DTE and change the visibility of Visual Studio. To hide VS change the SuppressUI property to $true
$dte = new-object -com $vsName
Start-Sleep -Seconds 20
$dte.SuppressUI = $false
$dte.MainWindow.Visible = $true

In [ ]:
# Create a new XAE project from a template
$sln = $dte.Solution
$project = $sln.AddFromTemplate($template,$projectDirectory,$projectName)
$systemManager = $project.Object

In [ ]:
# Create a new PLC project from a template
$plc = $systemManager.LookupTreeItem("TIPC")
$newProject = $plc.CreateChild($plcName, 0, "", $plcTemplate)

In [ ]:
# Edit the content of the MAIN program
$main = $systemManager.LookupTreeItem("TIPC^$plcName^$plcName Project^POUs^MAIN")
$main.DeclarationText =  @"
PROGRAM MAIN
VAR
    test2 : DINT;
END_VAR
"@

$main.ImplementationText = @"
test2 := test2 + 1;
"@

## Adding route by IP Address
The example below adds a route using Automation Interface. The same could be accomplished using the ADS Powershell module shown later.

In [ ]:
# Set the target

# Adding TwinCAT ADS Route
$localAmsNetId = Get-ItemPropertyValue -Path HKLM:\SOFTWARE\WOW6432Node\Beckhoff\TwinCAT3\System -Name "AmsNetId" 
$localAmsNetId = $localAmsNetId -join "."

$systemManager.SetTargetNetId($localAmsNetId)

$routes = $systemManager.LookupTreeItem("TIRR")

$xml = "<TreeItem>
    <RoutePrj>
        <TargetList>
            <Search>$targetIpAddress</Search>
        </TargetList>
    </RoutePrj>
</TreeItem>"
$routes.ConsumeXml($xml)
$result = $routes.ProduceXml()

$xmlDocument = [xml]$result
$remoteAmsNetId = Select-Xml -Xml $xmlDocument -XPath "//TreeItem/RoutePrj/TargetList/Target/IpAddr[text()=""$targetIpAddress""]/../NetId" | ForEach-Object {$_.node.InnerXML}
$remoteName = Select-Xml -Xml $xmlDocument -XPath "//TreeItem/RoutePrj/TargetList/Target/IpAddr[text()=""$targetIpAddress""]/../Name" | ForEach-Object {$_.node.InnerXML}

Write-Output "Found target at $targetIpAddress with hostname $remoteName and AmsNetId $remoteAmsNetId"

$xml = @"
<TreeItem>
    <ItemName>Route Settings</ItemName>
    <PathName>TIRR</PathName>
    <RoutePrj>
    <TargetList>
        <BroadcastSearch>false</BroadcastSearch>
    </TargetList>
    <AddRoute>
        <RemoteName>$remoteName</RemoteName>
        <RemoteNetId>$remoteAmsNetId</RemoteNetId>
        <RemoteIpAddr>$targetIpAddress</RemoteIpAddr>
        <UserName>Administrator</UserName>
        <Password>1</Password>
        <NoEncryption></NoEncryption>
        <LocalName>Temporary Route Local</LocalName>
    </AddRoute>
    </RoutePrj>
</TreeItem>
"@
$routes.ConsumeXml($xml)


Found target at 10.1.12.180 with hostname TESTPLC01 and AmsNetId 172.18.234.196.1.1


## Deploying project to remote PLC
The example below changes the real time settings of the PLC. You may need to alter depending on the hardware you are deploying to.

In [ ]:
# Overwrite the realtime settings to match the target
$xml = @"
<TreeItem>
    <RTimeSetDef>
        <MaxCPUs>4</MaxCPUs>
        <Affinity>#x8</Affinity>
        <CPUs>
            <CPU id="3">
                <LoadLimit>80</LoadLimit>
                <BaseTime>10000</BaseTime>
                <LatencyWarning>0</LatencyWarning>
            </CPU>
        </CPUs>
    </RTimeSetDef>
</TreeItem>
"@

Write-Output "Changing real time configuration to match build agent hardware"
$systemManager.LookupTreeItem("TIRS").ConsumeXml($xml)

Changing real time configuration to match build agent hardware


In [ ]:
# Note: if you have not activated on this target before, you will need to license it manually before continuing
$systemManager.SetTargetNetId($remoteAmsNetId)

Write-Output "Activating configuration"
$systemManager.ActivateConfiguration()

Write-Output "Restarting TwinCAT"
$systemManager.StartRestartTwinCAT()

Activating configuration
Restarting TwinCAT


## Online Changes

This example edits the main program and makes an online change.

In [ ]:
# Online change
# Edit the content of the MAIN program
$main.DeclarationText = @"
PROGRAM MAIN
VAR
    test5 : DINT;
END_VAR
"@

$main.ImplementationText = @"
test5 := test5 + 100;
"@

$plcProjectItem = $systemManager.LookupTreeItem("TIPC^$plcName^$plcName Project")

$compileResult = $plcProjectItem.CheckAllObjects()
Write-Output "PLC compiles Ok? $compileResult"

if ($compileResult)
{
    $plcProjectItem.Login($loginServiceFlags::CompileCodeBeforeLogin + $loginServiceFlags::SuppressAllDialogs)
    $plcProjectItem.Logoff()
}

PLC compiles Ok? True


## Powershell ADS

See https://download.beckhoff.com/download/document/automation/twincat3/TwinCAT_3_ADS_Powershell_Module_EN.pdf

Run `Install-Module -Name TcXaeMgmt` in a Powershell terminal first.


In [ ]:
$route = Get-AdsRoute -Address $remoteAmsNetId
$session = New-TcSession -Address $remoteAmsNetId -Port 851

In [ ]:
$session | Read-TcValue -Path "MAIN.test5"

66900


# Cleanup

In [ ]:
# $sln.Close()
# $dte.Quit()

# [EnvDteUtils.MessageFilter]::Revoke()

# Remove-Item -Path $targetDir -Recurse -Force